## HMMlearn 을 이용한 HMM 학습과 BKT


### 1) 라이브러리 불러오기

- `hmmlearn` 설치하기 : `pip install hmmlearn`
-  hmmlearn 0.2.8 
-  https://github.com/hmmlearn/hmmlearn 

In [1]:
import numpy as np
from hmmlearn import hmm

### 2) CategorialHMM 함수

- 관찰 변수 범주형(categorical) 변수인 경우 `CategoricalHMM()` 함수 사용

#### A. Parameter setting

In [2]:
#관찰변수와 잠재변수가 binary variable일 때 parameter setting
#initial probability
startprob=np.array([0.7, 0.3])

#transition matrix
transmat=np.array([[0.6, 0.4],
                  [0, 1]])
#emission matrix
emissionprob=np.array([[0.95, 0.05],
                  [0.1, 0.9]])


#### B. Model setting

-  `CategoricalHMM()` 함수의 모델 설정과 관련한 인수
    - `params=''`, `init_params=''`  이면 학습하지 않고 기존에 세팅한 모든 parameter를 그대로 fix한 채로 학습함
    - `params='ste'` 와 같이 설정하면 ste (t=transition, e=emission, s=start) 중 빠진 parameter는 fix되고 나머지만 데이터로부터 학습됨 
    - `n_components` : number of hidden states for latent variable (잠재변수의 범주 개수)
    
    
-  모델 변수 지정 후 parameter 등 property 
    - `n_features` : number of states for output variable (관찰변수의 범주 개수)
    - `startprob_` : start probability
    - `transmat_` : transition probability matrix
    - `emissionprob_` : emission probability matrix

In [3]:
#model setting
model = hmm.CategoricalHMM(n_components=2, params='', init_params='')  
model.n_features=2
model.startprob_ = startprob
model.transmat_ = transmat
model.emissionprob_ = emissionprob

#### C. Data generation and Model fitting
- model setting에 따라 data generation 후 학습(fitting)
- `sample()` method를 사용하면 model의 setting에 따라 데이터가 생성됨 
- `fit()` method를 setting 한 model에 적용하면 학습이 이루어지며 이때 `fit(data)`와 같이 사용하며 data는 관찰변수의 sequence인 (n, 1) array
- 위의 model setting에서 `params=''`, `init_params=''` 로 설정하였으므로 parameter는 실제 학습되지 않고 주어진 값을 그대로 고정하여 그 결과를 사용할 수 있게 한다. 


In [13]:
#data generation and model fitting

X, Z = model.sample(10) #generate a single sequence of a length based on HMM model
#X : 관찰변수의 sequence
#Z : 잠재변수의 true sequence

model.fit(X)  

CategoricalHMM(init_params='', n_components=2, params='',
               random_state=RandomState(MT19937) at 0x1D2EF393840)

In [14]:
#X는 (10,1) array
X     

array([[0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1]], dtype=int64)

In [15]:
#Z는 (1, 10) array
Z    

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

#### D. Prediction and Decoding
- `predict()` method는 (n, 1) array 형태의 관찰변수 sequence 를 주면 이로부터 각 hidden state를 예측한다. 이때 marginal probability $P(q_t=1|sequence)$ 를 계산하여 0.5가 넘으면 1로 판정한다. 
- `predict_proba()` method는 `predict()`를 적용하기 위해 계산하는 $P(q_t=1|sequence)$ 를 알려준다. 
- `decode()` method는 (n, 1) array 형태의 관찰변수 sequence 를 주면 joint probability($P(q_1, q_2, \cdots, q_t|sequence)$) 가 가장 높은 hidden state를 알려준다. output은 tuple 형태로 두 개의 변수에 저장하는데 앞의 값은 확률의 log값, 뒤의 값은 추정된 hidden state 이다. 
- `score()` method는 (n, 1) array 형태의 관찰변수 sequence 를 주면 이를 관찰할 joint probability($P(sequence)$)의 log 값을 계산해준다. 

In [17]:
#marginal probability 를 이용한 hidden state 추정
model.predict(X)

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1], dtype=int64)

In [18]:
#joint probability를 이용한 hidden state 추정
logp, state=model.decode(X)
print(logp)
print(state)

-6.296761798439953
[0 0 0 0 0 1 1 1 1 1]


- `fit()` 을 이용한 모델학습이 끝난 결과로부터 각각 `startprob_`, `transmat_` ,`emissionprob_` 의 값을 확인할 수 있다. 

In [12]:
#새로 학습되지 않고 처음 fix된 transition matrix 가 맞는지 확인한다. 
model.transmat_

array([[0.6, 0.4],
       [0. , 1. ]])

### 3) 예시 문제 풀기

- 예시에서  관찰변수 sequence가 $(𝑦_1,⋯,𝑦_4)=(0, 0, 1, 1)$ 로 주어졌을 때, 1), 2), 3), 4)를 구해보자. 

1)  $𝑃(𝑦_1=0,𝑦_2=0,𝑦_3=1,𝑦_4=1)$, 즉, 위의 관찰 결과를 얻을 확률(joint probability)을 구해보자. 


In [22]:
#score() 를 사용
y=np.array([[0],[0],[1],[1]])
logp=model.score(y) #log likelihood of X -> correct 
print(logp, np.exp(logp))

-1.8889991483194628 0.15122308499999998


2) $𝑃(𝑞_1,𝑞_2,𝑞_3,𝑞_4 |𝑦_1=0,𝑦_2=0,𝑦_3=1,𝑦_4=1)$ 가 가장 높은 $𝑞_1,𝑞_2,𝑞_3,𝑞_4$ 의 값은 어떤 경우인가?


In [23]:
logp, state=model.decode(y) #viterbi algoritm (분자 계산)
print(logp,np.exp(logp))
print(state)

-2.097098919669632 0.12281219999999998
[0 0 1 1]


3) $𝑃(𝑞_5=1 |𝑦_1=0,𝑦_2=0,𝑦_3=1,𝑦_4=1)$ 를 예측하여라.


In [24]:
#predict 함수는 marginal probability(P(q_i=1)) 을 계산해서 각 sequence의 state를 예측(가장 높은 확률의 state)  

model.predict(y)

array([0, 0, 1, 1], dtype=int64)

In [25]:
model.predict_proba(y) #predict marginal probability  

array([[0.98393102, 0.01606898],
       [0.84145278, 0.15854722],
       [0.02932677, 0.97067323],
       [0.00225591, 0.99774409]])

In [29]:
# P(q_4=1|[0, 0, 1, 1])과 P(q_4=0|[0, 0, 1, 1])과 transition probability 이용하여 계산
model.predict_proba(y)[3][0]*0.4+model.predict_proba(y)[3][1]

0.998646456657064

4) $𝑃(𝑦_5=1|𝑦_1=0,𝑦_2=0,𝑦_3=1,𝑦_4=1)$ 를 예측하여라. 

In [33]:
#위의 결과와 emission probability를 이용하여 계산
p=model.predict_proba(y)[3][0]*0.4+model.predict_proba(y)[3][1]
0.9*p+0.05*(1-p)

0.8988494881585044

### 4) Data로 부터 학습하기

- `CategoricalHMM()` 함수를 이용해 실제 sequence 데이터를 주고 parameter들을 학습시킬 수 있다. 
- `hmm.CategoricalMHH(n_componetns=2).fit(X, lengths)`: 여러 sample의 sequence(multiple sequence)를 합쳐서 X를 (n, 1) array로 만들어 학습시키면서 동시에 각 sequence의 길이를 legnths 인수로 알려준다. 
- 이 방법은 길이가 다른 sequence 들을 합쳐서 사용가능하게 해준다. 


#### A. 데이터가 주어졌을 때 

In [97]:
#주어진 데이터 (10명의 데이터, 4 step)
l_seq=4
n_sample=10

#Xseq 은 (n_sample*l_seq, 1) 의 모양임
Xseq=np.array([[0],[0],[1],[1],
             [0],[0],[1],[1],
             [0],[0],[0],[1],
             [0],[0],[0],[1],
             [0],[0],[1],[1],
             [0],[1],[1],[1],
             [0],[1],[1],[1],
             [1],[0],[1],[1],
             [1],[1],[1],[1],
             [1],[1],[1],[1]]) 



In [98]:
#model fiting ( one time -local optimization)
#random initial point에서 진행됨
model1 = hmm.CategoricalHMM(n_components=2)  
model1.n_features=2
model1.fit(Xseq,[l_seq]*n_sample)

CategoricalHMM(n_components=2,
               random_state=RandomState(MT19937) at 0x1D2EF393840)

In [114]:
#Convergence monitoring
model1.monitor_

ConvergenceMonitor(
    history=[-35.55174456069207, -25.284182779027006, -23.66822273092971, -21.73669696731069, -20.30780388137565, -19.68895682114933, -19.476790053308417, -19.40274805940719, -19.372709317296042, -19.354516310641003],
    iter=10,
    n_iter=10,
    tol=0.01,
    verbose=False,
)

In [110]:
#Initial probability learned
model1.startprob_

array([0.72358073, 0.27641927])

In [111]:
#transition probability learned
model1.transmat_

array([[4.82675332e-01, 5.17324668e-01],
       [4.73177321e-06, 9.99995268e-01]])

In [112]:
#emission probability learned
model1.emissionprob_

array([[0.96612727, 0.03387273],
       [0.05880891, 0.94119109]])

In [160]:
#학습을 반복해서 배스트 모델을 찾기
# repeating and finding the best results (random initialization)

best_score = best_model = None
n_fits = 1000
for i in range(n_fits):
    modeleach = hmm.CategoricalHMM(n_components=2)
    modeleach.fit(Xseq,[l_seq]*n_sample)
    score = modeleach.score(Xseq)
    #print(f'Model #{i+1}\tScore: {score}')
    if best_score is None or score > best_score:
        best_model = modeleach
        best_score = score

In [161]:
best_score

-23.943035084614134

In [162]:
best_model.startprob_.round(3)

array([0.318, 0.682])

In [163]:
best_model.transmat_.round(3)

array([[1.   , 0.   ],
       [0.109, 0.891]])

In [164]:
best_model.emissionprob_.round(3)

array([[0.151, 0.849],
       [0.536, 0.464]])

#### B. 데이터를 생성하고 학습해 보기

In [165]:
# multi-sequence 가상 데이터를 생성하려면 다음과 같이 한다.
#HMM setting for data generation of multiple sequences 

#HMM parameter setting
startprob=np.array([0.7, 0.3])
transmat=np.array([[0.6, 0.4],
                  [0, 1]])
emissionprob=np.array([[0.95, 0.05],
                  [0.1, 0.9]])

#model setting 
model = hmm.CategoricalHMM(n_components=2, params='', init_params='')  
model.n_features=2
model.startprob_ = startprob
model.transmat_ = transmat
model.emissionprob_ = emissionprob

#data generation
n_sample=10000   #n_sample : sample size (sequence의 개수)
l_seq=10       #l_seq : sequence의 time step  개수
Xseq=np.zeros(shape=(n_sample*l_seq, 1), dtype=int) #최종 multi sequence data를 저장할 변수
Zseq=np.zeros(shape=(n_sample*l_seq, ), dtype=int)
for i in range(n_sample):
    X, Z = model.sample(l_seq) 
    Xseq[(i*l_seq):(i+1)*l_seq]=X
    Zseq[(i*l_seq):(i+1)*l_seq]=Z
    #if i <10 : 
    #    print(X)
    #    print(Z)

In [166]:
#model fiting ( one time -local optimization)
#random initial point에서 진행됨
model2 = hmm.CategoricalHMM(n_components=2)  
model2.n_features=2
model2.fit(Xseq,[l_seq]*n_sample)

CategoricalHMM(n_components=2,
               random_state=RandomState(MT19937) at 0x1D2EF393840)

In [167]:
#Convergence monitoring
model2.monitor_

ConvergenceMonitor(
    history=[-78052.02352132813, -56026.50242196378, -55957.89122569403, -55919.073405796604, -55886.73363097737, -55855.27577544248, -55823.70994643497, -55792.22535953317, -55761.25784469623, -55731.22014722861],
    iter=10,
    n_iter=10,
    tol=0.01,
    verbose=False,
)

In [168]:
#Initial probability learned
model2.startprob_

array([0.13268155, 0.86731845])

In [169]:
#transition probability learned
model2.transmat_

array([[0.1866936 , 0.8133064 ],
       [0.95388712, 0.04611288]])

In [170]:
#emission probability learned
model2.emissionprob_

array([[0.19961316, 0.80038684],
       [0.30182398, 0.69817602]])

In [171]:
#최종 loss function 값
model2.score(Xseq)

-56332.88675187469

In [172]:
#학습을 반복해서 배스트 모델을 찾기
# repeating and finding the best results (random initialization)

best_score = best_model = None
n_fits = 2000
for i in range(n_fits):
    modeleach = hmm.CategoricalHMM(n_components=2)
    modeleach.fit(Xseq,[l_seq]*n_sample)
    score = modeleach.score(Xseq)
    print(f'Model #{i+1}\tScore: {score}')
    if best_score is None or score > best_score:
        best_model = modeleach
        best_score = score

Model #1	Score: -56071.50405763127
Model #2	Score: -55541.012684816604
Model #3	Score: -55527.80456797091
Model #4	Score: -57645.09402203622
Model #5	Score: -59133.80497400741
Model #6	Score: -56443.0903945478
Model #7	Score: -56706.44085875095
Model #8	Score: -56309.19012335569
Model #9	Score: -56632.58935271051
Model #10	Score: -54649.85245271422
Model #11	Score: -55226.70251403452
Model #12	Score: -55706.138587900736
Model #13	Score: -56301.188901545036
Model #14	Score: -60367.33981529027
Model #15	Score: -55218.0458011816
Model #16	Score: -57845.13648940478
Model #17	Score: -59234.08409606974
Model #18	Score: -57899.92691813912
Model #19	Score: -54826.18119449448
Model #20	Score: -56338.89960001078
Model #21	Score: -56729.90229157405
Model #22	Score: -56602.97038791846
Model #23	Score: -56719.57407347846
Model #24	Score: -58288.43967739815
Model #25	Score: -59357.92387931747
Model #26	Score: -55743.09537055125
Model #27	Score: -55781.898129080684
Model #28	Score: -56526.32980234438

Model #225	Score: -55088.52387469561
Model #226	Score: -57125.12395957391
Model #227	Score: -56514.58649106818
Model #228	Score: -56114.761732621286
Model #229	Score: -63073.46139225535
Model #230	Score: -54106.02754859992
Model #231	Score: -58797.2198241777
Model #232	Score: -68963.53894046898
Model #233	Score: -55202.23501632577
Model #234	Score: -60084.00721855096
Model #235	Score: -58322.835595227116
Model #236	Score: -55660.73512783146
Model #237	Score: -54359.48905308987
Model #238	Score: -61682.81835119832
Model #239	Score: -58392.40097353658
Model #240	Score: -56074.43532223011
Model #241	Score: -57927.82350591309
Model #242	Score: -55079.139197119446
Model #243	Score: -64832.826887893534
Model #244	Score: -59586.706486374205
Model #245	Score: -56568.348237319966
Model #246	Score: -56135.19462574999
Model #247	Score: -53688.11401685112
Model #248	Score: -57571.2827494252
Model #249	Score: -60179.0692960717
Model #250	Score: -59543.82870330093
Model #251	Score: -56466.7757540833

Model #446	Score: -56681.83152156538
Model #447	Score: -54082.60632905637
Model #448	Score: -56574.76849691731
Model #449	Score: -57798.38137138294
Model #450	Score: -60107.27474099731
Model #451	Score: -54254.71706230561
Model #452	Score: -56149.00814102015
Model #453	Score: -55572.487680084196
Model #454	Score: -57015.67257909641
Model #455	Score: -55740.46902000951
Model #456	Score: -59941.61650120631
Model #457	Score: -54739.77084602406
Model #458	Score: -55428.13366007939
Model #459	Score: -56098.04347335967
Model #460	Score: -55101.36157062467
Model #461	Score: -56248.76203355508
Model #462	Score: -56355.20772677359
Model #463	Score: -55258.50910582265
Model #464	Score: -56450.42412065242
Model #465	Score: -64527.81898269888
Model #466	Score: -56346.68487156028
Model #467	Score: -56311.60111765865
Model #468	Score: -60089.18563214491
Model #469	Score: -55152.16857104835
Model #470	Score: -57903.864588151
Model #471	Score: -60163.07100825318
Model #472	Score: -57256.83476601004
Mo

Model #667	Score: -55856.89482960426
Model #668	Score: -56775.15344882214
Model #669	Score: -61774.69173873404
Model #670	Score: -56158.46899985927
Model #671	Score: -56190.91839713352
Model #672	Score: -56703.73106771803
Model #673	Score: -56234.64725030709
Model #674	Score: -56421.91865983214
Model #675	Score: -55060.44332013072
Model #676	Score: -58543.38093037439
Model #677	Score: -56260.05627542844
Model #678	Score: -54132.50827505414
Model #679	Score: -57423.40400082283
Model #680	Score: -55088.93228762128
Model #681	Score: -56092.330219411066
Model #682	Score: -59010.90839335933
Model #683	Score: -56101.526195446204
Model #684	Score: -56129.68791632952
Model #685	Score: -54731.88877117955
Model #686	Score: -58268.03061424554
Model #687	Score: -59666.62434602788
Model #688	Score: -55975.84619832219
Model #689	Score: -56156.883332366924
Model #690	Score: -54774.18938883415
Model #691	Score: -56614.126005533486
Model #692	Score: -56206.48439465264
Model #693	Score: -54887.111773654

KeyboardInterrupt: 

In [173]:
best_score

-53685.79894358745

In [174]:
best_model.startprob_.round(3)

array([0.995, 0.005])

In [175]:
best_model.transmat_.round(3)

array([[0.637, 0.363],
       [0.167, 0.833]])

In [176]:
best_model.emissionprob_.round(3)

array([[0.546, 0.454],
       [0.01 , 0.99 ]])